# updating all dataset files

In [21]:
import os
# import pymongo
import json
from nltk.sentiment import SentimentIntensityAnalyzer

### directory path

In [22]:
homePath = "C:/Hammad Aslam/BS IT (post ADP)/3rd Semester/Capstone Project/Project/backend/datasets/categories/allFiles"

### calculating intensity and topics relevance scores

In [23]:
def calculate_text_analysis_score(review_text, topics):
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(review_text)
    # Adjust sentiment score based on relevance of topics
    topic_relevance = sum([review_text.lower().count(topic.lower()) for topic in topics])
    adjusted_score = sentiment_scores['compound'] + (0.1 * topic_relevance)  # Adjust sentiment score based on topic relevance
    return adjusted_score

### updates the data with sentiment scores and helpfulness scores

In [24]:
def setUpdatedValues(data):
    
    for product in data:
        for review in product['reviews']:
            text_analysis_score = calculate_text_analysis_score(review['review_body'], review['review_topics'])
            # print(text_analysis_score)
            helpfulness_score = (0.2 * float(review['review_rating'])) + (0.4 * text_analysis_score) + (0.4 * int(review['review_votes']))
            # Normalize the helpfulness score to range from 0 to 1
            min_score = min(float(review['review_rating']), -1) * 0.2 + min(text_analysis_score, -1) * 0.4 + min(int(review['review_votes']), 0) * 0.4

            max_score = max(float(review['review_rating']), 1) * 0.2 + max(text_analysis_score, 1) * 0.4 + max(int(review['review_votes']), 1) * 0.4

            normalized_score = (helpfulness_score - min_score) / (max_score - min_score)
            
            review['review_helpfulness'] = normalized_score
            
            sia = SentimentIntensityAnalyzer()
            sentiment_scores = sia.polarity_scores(review['review_body'])
            review['sentiment'] = "Positive" if sentiment_scores['compound'] >= 0.5 else "Negative"
    
    return data

### reads the files and punches the updated data in the file

In [25]:
def readFiles(file):
    with open(file, "r") as f:
        data = json.load(f)
    
    newData = setUpdatedValues(data)
    with open(file, 'w') as json_file:
        json.dump(newData, json_file, indent=4)
    

### accessing each file exactly once in the directory

In [26]:
count = 0
for filename in os.listdir(homePath):
    if filename.endswith(".json"):
        file = homePath + "/" + filename
        readFiles(file)
        count += 1
        print(f"{count} done")

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
